In [1]:
import numpy as np
import random
import math

In [2]:
LIMITS = 100
DIMENSION = 30
MAX_FITNESS = 1
MAX_ITER = 100
SAMPLE_SIZE = 1
INITIAL_SAMPLE_SIZE = 50

In [3]:
# Diccionario para no recalcular fitness:
dic = {}

In [4]:
def esfera(sol):
    sum = 0
    for xi in sol:
        sum += xi ** 2
    return sum

In [5]:
def fitness(sol):
    maxi = DIMENSION * LIMITS * LIMITS
    assert(maxi > 0)
    esf = esfera(sol)
    assert(esf <= maxi)
    return (maxi - esf) / maxi

In [6]:
def f(sol):
    fit = dic.get(tuple(sol))
    if fit == None:
        fit = fitness(sol)
        dic[tuple(sol)] = fit
    print(f"FITNESS de {sol} = {fit}")
    return fit

In [7]:
def manhattan_distance(sol1, sol2):
    n = len(sol1)
    assert(n == len(sol2))
    return sum([abs(sol1[i] - sol2[i]) for i in range(n)])

In [8]:
def euclidean_distance(sol1, sol2):
    n = len(sol1)
    assert(n == len(sol2))
    return math.sqrt(sum([(sol1[i] - sol2[i]) ** 2 for i in range(n)]))

In [9]:
def hamming_distance(sol1, sol2):
    n = len(sol1)
    #print(f"sol1 = {sol1}\nsol2 = {sol2}")
    assert(n == len(sol2))
    return [sol1[i] == sol2[i] for i in range(n)].count(False)

In [10]:
def inverse_distance(sol1, sol2, dist):
    d = dist(sol1, sol2)
    assert(d > 0) # Creo que todo está preparado para no samplear dos iguales.
    return 1 / d if d != 0 else 2 # Realmente la forma de manejar el infinito depende de la distancia
# El 2 funciona para Hammings porque no puede haber distancias d 0 < d < 1 (es decir, tales que 1 / d > 2).

In [11]:
inverse_distance([1,0,1,0], [0,1,0,1], hamming_distance)

0.25

In [12]:
def w(sol1, sol2, A, dist):
    assert(len(sol1) == len(sol2))
    sum = 0
    for sol in A:
        assert(len(sol) == len(sol1))
        sum += inverse_distance(sol1, sol, dist)
    assert(sum > 0) # Creo que se puede demostrar que esto no va a ser 0.
    inv = inverse_distance(sol1, sol2, dist)
    res = inv / sum
    #print(f"inv: {inv}\nsum: {sum}\nres: {res}")
    return res

In [13]:
def g(sol, A, dist):
    sum = 0
    for point in A:
        wei = w(sol, point, A, dist)
        fit = f(point)
        #print(f"\nwei: {wei} \n fit: {fit}")
        sum += wei * fit
    print(f"\n\n g: {sum}\n\n")
    return sum

In [14]:
def sample(k, A, dist, skip = []):
    n = DIMENSION
    sample = []
    while len(sample) < k:
        x = [random.randint(0, LIMITS) for _ in range(n)]
        if x in sample or x in skip:
            continue
        #y = random.randint(0, MAX_FITNESS)
        y = random.uniform(0, MAX_FITNESS)
        g_val = g(x, A, dist)
        if g_val > y:
            #print(fitness(x))
            print(f"Muestra añadida {x}\ncon valor de g: {g_val} y valor de y: {y}")
            sample.append(x)
    return sample

In [15]:
def init(k):
    n = DIMENSION
    init = []
    while len(init) < k:
        sol = [random.randint(0, LIMITS) for _ in range(n)]
        if sol not in init: # Esto se evita usando un set. Aún no tengo claro si necesito índices, o si el set es eficiente en python.
            init.append(sol)
    return init

def max_k_fitness(set_sols, k):
    sor = sorted([(fitness(elem), elem) for elem in set_sols])
    return [elem for (_,elem) in sor][-k:]

def max_fitness(set_sols):
    max_value = fitness(set_sols[0])
    max_elem = set_sols[0]
    for elem in set_sols[1:]:
        f = fitness(elem)
        if f > max_value:
            max_value = f
            max_elem = elem
    return max_elem, max_value

def next_gen(curr_gen, k):
    new = sample(k, skip = curr_gen)
    return max_k_fitness(curr_gen + new, k)

def algorithm():
    gen = init(SAMPLE_SIZE)
    for _ in range(MAX_ITER):
        gen = next_gen(gen, SAMPLE_SIZE)
    return max_fitness(gen)

In [16]:
def max_f(A):
    assert(len(A) > 0)
    max_value = f(A[0])
    max_elem = A[0]
    for elem in A[1:]:
        fit = f(elem)
        if fit > max_value:
            max_value = fit
            max_elem = elem
    return max_elem, max_value

In [17]:
def alg(sample_size, num_iter, initial_sample_size, dist):
    dic.clear()
    A = init(initial_sample_size)
    for i in range(num_iter):
        print(f"iter {i}")
        A.extend(sample(sample_size, A, dist, skip = A))
    return max_f(A)

In [18]:
elem, fit = alg(sample_size = SAMPLE_SIZE, num_iter = MAX_ITER, initial_sample_size = INITIAL_SAMPLE_SIZE, dist = euclidean_distance)
print(f'Máximo valor de fitness ({fit}) encontrado para {elem}, con volumen {esfera(elem)}.')

iter 0
FITNESS de [74, 49, 30, 72, 61, 40, 92, 49, 3, 14, 83, 86, 94, 52, 77, 86, 51, 98, 33, 95, 32, 4, 77, 91, 31, 56, 40, 88, 16, 46] = 0.5903866666666666
FITNESS de [71, 87, 99, 40, 10, 64, 13, 28, 62, 34, 28, 17, 51, 91, 3, 72, 96, 3, 11, 57, 16, 44, 99, 37, 9, 42, 65, 51, 46, 7] = 0.70633
FITNESS de [55, 1, 73, 71, 50, 69, 16, 88, 95, 47, 79, 51, 75, 42, 5, 100, 29, 64, 10, 90, 68, 83, 48, 80, 38, 6, 46, 82, 52, 31] = 0.6217133333333333
FITNESS de [10, 10, 43, 33, 21, 79, 27, 86, 82, 54, 20, 50, 28, 66, 20, 83, 9, 76, 5, 62, 82, 10, 3, 40, 22, 63, 100, 81, 72, 55] = 0.6990133333333334
FITNESS de [89, 100, 92, 56, 9, 69, 1, 97, 44, 57, 73, 20, 5, 15, 3, 46, 47, 39, 89, 27, 94, 93, 63, 75, 44, 48, 66, 68, 80, 81] = 0.5909933333333334
FITNESS de [47, 15, 56, 28, 80, 65, 83, 51, 87, 95, 32, 98, 77, 5, 29, 2, 54, 76, 56, 89, 74, 32, 34, 69, 17, 81, 21, 49, 88, 87] = 0.6077966666666667
FITNESS de [68, 9, 38, 23, 40, 50, 25, 22, 66, 12, 72, 34, 29, 8, 68, 19, 79, 8, 2, 41, 30, 36, 26, 5